In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_29/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Naive Bayes

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Problema

En la clase vamos a construir un clasificador que intente predecir si una persona ganará más de US$ 50.000 por año 

En esta notebook vamos a preparar los datos para ser utilizados en el entrenamiento y testing del modelo



## Dataset

https://archive.ics.uci.edu/ml/datasets/Adult

Los datos corresponden a un censo de 1994.

Los campos son

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-

op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, 
Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.


In [3]:
data_location = '../Data/adult.data'

data = pd.read_csv(data_location, header=None, sep=',')

columns_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

data.columns = columns_names

data.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


Vemos si tiene registros con campos nulos:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Ejercicio 1 - Limpieza

¿Cuáles son las variables categoricas, y cuáles las numéricas?

Eliminemos en el dataset los espacios que rodean las variables categóricas usando el método `Series.str.strip`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.strip.html

In [5]:
categorical_columns = [col for col in data.columns if data[col].dtypes == 'object']

categorical_columns

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'income']

In [6]:
numerical_columns = [col for col in data.columns if data[col].dtypes != 'object']

numerical_columns

['age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week']

In [7]:
for col in categorical_columns:
    data[col] = data[col].str.strip()    

## Ejercicio 2 - Limpieza (cont.)

Los valores faltantes en este dataset aparecen codificados como '?'

Reemplacemos los '?' por NaN donde corresponda. Para ello usemos el método `replace`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

¿Qué columnas deldataset quedaron con nulos después de este reemplazo?

In [8]:
data_with_NaN = data.replace(to_replace='?', value = np.NaN)

In [9]:
cant_filas = data.shape[0]
cant_filas

32561

In [10]:
for column in data_with_NaN.columns:
    if any(data_with_NaN[column].isna()):
        print(column)

workclass
occupation
native_country


In [11]:
# verificamos que esas columnas tienen valores nulos:
data_with_NaN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Ejercicio 3 - Drop null

Eliminemos los registros incompletos del dataset.


In [12]:
data_complete = data_with_NaN.dropna()
data_complete.shape

(30162, 15)

## Ejercicio 4 - Features, Target

Construyamos una matriz de features (X) y el vector target (Y) para predecir `income` en el dataset de datos completos

¿Qué valores toma la variable `income` en el dataset?


In [13]:
data_complete.income.value_counts()

<=50K    22654
>50K      7508
Name: income, dtype: int64

In [14]:
data_complete.income.unique()

array(['<=50K', '>50K'], dtype=object)

In [15]:
X = data_complete.drop(['income'], axis=1)

Y = data_complete['income']

## Ejercicio 5 - Train Test Split

Constuyamos los conjuntos de train y test, asignando el 70% de los registros a train y el 30% a test

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1237)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(21113, 14)
(9049, 14)
(21113,)
(9049,)


In [17]:
X_train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country'],
      dtype='object')

## Ejercicio 6 - Feature Engineering (Categóricas)

Usemos `OneHotEncoder` para transformar las variables categóricas del dataset.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features

El resultado de este ejercicio deben ser dos instancias de DataFrame: X_train_categorical y X_test_categorical, que tengan las variables categóricas transformadas para los datos de train y test respectivamente.

In [18]:
encoder_categories = []

X_categorical_columns = [x for x in categorical_columns if x != 'income']

for col in X_categorical_columns:    
    col_categories = data_complete[col].unique()
    encoder_categories.append(col_categories)

encoder_categories

[array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
        'Local-gov', 'Self-emp-inc', 'Without-pay'], dtype=object),
 array(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', '7th-8th', 'Doctorate', 'Assoc-voc', 'Prof-school',
        '5th-6th', '10th', 'Preschool', '12th', '1st-4th'], dtype=object),
 array(['Never-married', 'Married-civ-spouse', 'Divorced',
        'Married-spouse-absent', 'Separated', 'Married-AF-spouse',
        'Widowed'], dtype=object),
 array(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners',
        'Prof-specialty', 'Other-service', 'Sales', 'Transport-moving',
        'Farming-fishing', 'Machine-op-inspct', 'Tech-support',
        'Craft-repair', 'Protective-serv', 'Armed-Forces',
        'Priv-house-serv'], dtype=object),
 array(['Not-in-family', 'Husband', 'Wife', 'Own-child', 'Unmarried',
        'Other-relative'], dtype=object),
 array(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo

In [19]:
encoder = OneHotEncoder(categories = encoder_categories, sparse=False)

encoder = encoder.fit(X_train[X_categorical_columns])

# ver que pasa si uso 
#print(encoder.get_feature_names())
#print(encoder.get_feature_names(X_categorical_columns))
    
X_train_encoded = encoder.transform(X_train[X_categorical_columns])
X_train_categorical = pd.DataFrame(X_train_encoded, columns = encoder.get_feature_names(X_categorical_columns))
#X_train_categorical.sample(5)

X_test_encoded = encoder.transform(X_test[X_categorical_columns])
X_test_categorical = pd.DataFrame(X_test_encoded, columns = encoder.get_feature_names(X_categorical_columns))
X_test_categorical.head()

,workclass_State-gov,workclass_Self-emp-not-inc,workclass_Private,workclass_Federal-gov,workclass_Local-gov,workclass_Self-emp-inc,workclass_Without-pay,education_Bachelors,education_HS-grad,education_11th,...,native_country_Outlying-US(Guam-USVI-etc),native_country_Scotland,native_country_Trinadad&Tobago,native_country_Greece,native_country_Nicaragua,native_country_Vietnam,native_country_Hong,native_country_Ireland,native_country_Hungary,native_country_Holand-Netherlands
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
X_train.shape

(21113, 14)

In [21]:
X_train_categorical.shape

(21113, 98)

In [22]:
X_train_encoded

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

## Ejercicio 7 -  Feature Engineering (Continuas)

Usemos `RobustScaler` para transformar las variables numéricas del dataset.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html

https://scikit-learn.org/stable/modules/preprocessing.html

*This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).*

El resultado de este ejercicio deben ser dos instancias de DataFrame: X_train_numerical y X_test_numerical, que tengan las variables numéricas transformadas para los datos de train y test respectivamente.


In [23]:
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_train_numerical = pd.DataFrame(X_train_scaled, columns = numerical_columns)
#X_train_numerical.head()

X_test_scaled = scaler.transform(X_test[numerical_columns])
X_test_numerical = pd.DataFrame(X_test_scaled, columns = numerical_columns)
X_test_numerical.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,-0.736842,0.438467,0.00,0.0,0.0,0.0
1,0.000000,0.057352,0.00,0.0,0.0,3.0
2,-0.368421,-0.801619,-0.25,0.0,0.0,0.0
3,-0.473684,-0.906570,-0.25,0.0,0.0,-0.4
4,-0.736842,-0.065428,0.00,0.0,0.0,1.0


In [24]:
X_train_numerical.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,-1.000000,-1.111697,0.00,0.0,0.0,-5.0
1,-0.736842,1.477544,-0.25,0.0,0.0,0.0
2,1.157895,0.436143,-0.25,0.0,625.0,-0.4
3,0.578947,-0.495027,0.25,0.0,0.0,3.2
4,0.368421,-0.021745,-0.75,5178.0,0.0,-0.8


## Ejercicio 8 - Combinación de los datos transformados

Combinemos los datos de train conformados por X_train_categorical, X_train_numerical, Y_train en una instancia de DataFrame, y guardemos ese DataFrame en el archivo **adult_train.csv** con campos separados por tab.

Lo mismo para los datos de test, conformados por X_test_categorical, X_test_numerical, Y_test en el archivo **adult_test.csv**

Ayuda: usar shape para ver las dimensiones de los datasets a combinar y las dimensiones de los resultados.

In [25]:
Y_train_concat = Y_train.reset_index(drop=True)
Y_test_concat = Y_test.reset_index(drop=True)

data_train = pd.concat([X_train_categorical, X_train_numerical, Y_train_concat], axis=1)

data_test = pd.concat([X_test_categorical, X_test_numerical, Y_test_concat], axis=1)

data_train_out_file_path = '../Data/adult_train.csv'
data_test_out_file_path = '../Data/adult_test.csv'

data_train.to_csv(data_train_out_file_path, sep='\t', index=False)
data_test.to_csv(data_test_out_file_path, sep='\t', index=False)

In [26]:
# sin hacer reset_idex():
X_train_categorical.index

RangeIndex(start=0, stop=21113, step=1)

In [27]:
X_train_numerical.index

RangeIndex(start=0, stop=21113, step=1)

In [28]:
Y_train.index

Int64Index([ 1701, 15819, 10252, 29996, 17206,  2770, 23220, 26972, 11873,
             6759,
            ...
            13529, 12535,  3102, 24305,  9290, 26090, 21798, 27169,  1013,
             2520],
           dtype='int64', length=21113)

In [29]:
Y_train_concat.index

RangeIndex(start=0, stop=21113, step=1)

In [30]:
Y_train.index

Int64Index([ 1701, 15819, 10252, 29996, 17206,  2770, 23220, 26972, 11873,
             6759,
            ...
            13529, 12535,  3102, 24305,  9290, 26090, 21798, 27169,  1013,
             2520],
           dtype='int64', length=21113)